# Postprocess the raw simulation data

Go through all subdirectories with name `out/T_*`, and use simulation data to compute overview of Mr, Hc and BHmax as function of temperature. Compute Hc, Mr, and BHmax at this point.

In [35]:
import pathlib
import mammos_entity as me
import mammos_analysis
import json
import pandas as pd


In [36]:
Ts = []
Hc = []
Mr = []
BHmax = []

In [37]:
runs = list(pathlib.Path("./out").glob(pattern="T_*"))

In [38]:
runs

[PosixPath('out/T_106.15664428351349'),
 PosixPath('out/T_0.0'),
 PosixPath('out/T_530.7832214175675'),
 PosixPath('out/T_955.4097985516214'),
 PosixPath('out/T_743.0965099845944'),
 PosixPath('out/T_371.5482549922972'),
 PosixPath('out/T_477.7048992758107'),
 PosixPath('out/T_583.8615435593242'),
 PosixPath('out/T_849.2531542681079'),
 PosixPath('out/T_636.9398657010809'),
 PosixPath('out/T_796.1748321263511'),
 PosixPath('out/T_1008.4881206933782'),
 PosixPath('out/T_690.0181878428376'),
 PosixPath('out/T_53.07832214175674'),
 PosixPath('out/T_265.39161070878373'),
 PosixPath('out/T_159.23496642527022'),
 PosixPath('out/T_424.62657713405395'),
 PosixPath('out/T_318.46993285054043'),
 PosixPath('out/T_902.3314764098646'),
 PosixPath('out/T_212.31328856702697')]

In [39]:
for run in runs:
    print(f"Working on {run=}")
    T = float(str(run).split("T_")[1])
    try:
        df = pd.read_csv(run/"hystloop.dat", sep='\t')
    except FileNotFoundError:
        # if material is too soft, hystloop.dat is not written
        print(f"No hystloop.dat found for {T=}")
        continue

    file = run / "parameters.json"
    # sanity check: is the temperature that the simulation saved the same as in the directory name?
    with open(file) as handle:
        param_dict = json.load(handle)
        assert param_dict["T"] == T

    # compute extrinsic parameters      
    H = df['H (A/m)'].values
    M = df['M (A/m)'].values
    res = mammos_analysis.hysteresis.extrinsic_properties(H=me.H(H), M=me.M(M), demagnetization_coefficient=1/3)
    
    # collect extrinsic parameters
    Ts.append(T)
    Hc.append(res.Hc.value)
    Mr.append(res.Mr.value)
    BHmax.append(res.BHmax.value)


Working on run=PosixPath('out/T_106.15664428351349')
Working on run=PosixPath('out/T_0.0')
Working on run=PosixPath('out/T_530.7832214175675')
Working on run=PosixPath('out/T_955.4097985516214')
Working on run=PosixPath('out/T_743.0965099845944')
Working on run=PosixPath('out/T_371.5482549922972')
Working on run=PosixPath('out/T_477.7048992758107')
Working on run=PosixPath('out/T_583.8615435593242')
Working on run=PosixPath('out/T_849.2531542681079')
Working on run=PosixPath('out/T_636.9398657010809')
Working on run=PosixPath('out/T_796.1748321263511')
Working on run=PosixPath('out/T_1008.4881206933782')
Working on run=PosixPath('out/T_690.0181878428376')
Working on run=PosixPath('out/T_53.07832214175674')
Working on run=PosixPath('out/T_265.39161070878373')
Working on run=PosixPath('out/T_159.23496642527022')
Working on run=PosixPath('out/T_424.62657713405395')
Working on run=PosixPath('out/T_318.46993285054043')
Working on run=PosixPath('out/T_902.3314764098646')
Working on run=Posix

In [40]:
# put data into DataFrame for easy sorting according to temperature T
parameters_df = pd.DataFrame(
    {
        "T": Ts,
        "Hc": Hc,
        "Mr": Mr,
        "BHmax": BHmax,
    }
)

parameters_df.sort_values("T", inplace=True, ignore_index=True)

In [41]:
## Write data to disk as entity collection.
me.io.entities_to_file(
    "hystloop_extrinsic_properties.csv",
    "These extrinsic properties were calculated using mammos-mumag simulations with 300 steps.",
    T=me.T(parameters_df["T"]),
    Hc=me.Hc(parameters_df["Hc"]),
    Mr=me.Mr(parameters_df["Mr"]),
    BHmax=me.BHmax(parameters_df["BHmax"]),
)

In [42]:
mammos_analysis.__version__

'0.4.0'